In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim

mapping = {'verified':0,'reviewTime':1,'reviewerID':2,'asin':3,"reviewText":4,"summary":5,"unixReviewTime":6,"sentiment":7,"id":8}
ls = []

for line in open('Data/music_reviews_train.json'):
    review_data = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data:
        if key in mapping:
            if key == "sentiment":
                
                tmp[mapping[key]] = 1 if review_data[key] == "positive" else 0
            else:
                tmp[mapping[key]] = review_data[key]
    ls.append(tmp)
#df = pd.read_json('Data/music_reviews_train.json', typ='series', orient='records')

In [2]:
ls_dev = []
for line in open('Data/music_reviews_dev.json'):
    review_data_dev = json.loads(line)
    tmp = [None]*len(mapping)
    for key in review_data_dev:
        if key in mapping:
            if key == "sentiment":
                
                tmp[mapping[key]] = 1 if review_data_dev[key] == "positive" else 0
            else:
                tmp[mapping[key]] = review_data_dev[key]
    ls_dev.append(tmp)

In [3]:
df = pd.DataFrame(ls)
df = df.drop(df[df[4].isna()].index) #Drops all None values for reviewtext

#DEV
df_dev = pd.DataFrame(ls_dev)
df_dev = df_dev.drop(df_dev[df_dev[4].isna()].index) #Drops all None values for reviewtext

In [4]:
mp = {value:key for key, value in mapping.items()}
df = df.rename(columns=mp)
target = np.array(df["sentiment"])

#DEV
df_dev = df_dev.rename(columns=mp)
target_dev = np.array(df_dev["sentiment"])

In [5]:
from collections import Counter

def oneHot(sentence):
    output = [0]*len(vocab)
    for word in sentence:
        if word in vocab:
            output[words2idx[word]] = 1
    return output

freqs = defaultdict(int)

for line in df["reviewText"]:
    for word in line.split():
        freqs[word] +=1

words = Counter(freqs)
w = words.most_common(1000)
vocab = set([x[0] for x in w])
words2idx = {x:idx for idx,x in enumerate(vocab)}
X = []

for line in df["reviewText"]:
    X.append(oneHot(line.split()))

In [6]:
# DEV
X_dev = []
for line in df_dev["reviewText"]:
    X_dev.append(oneHot(line.split()))

In [7]:
from sklearn.svm import SVC
clf = SVC(verbose=True,max_iter=100)
clf.fit(X, target)

[LibSVM]

C:\Users\Johan\Anaconda3\lib\site-packages\sklearn\svm\_base.py:249: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(max_iter=100, verbose=True)

In [8]:
clf.score(X,target)

0.5730494467012187

In [9]:
clf.score(X_dev,target_dev)

0.5414165666266506

In [10]:
sum(df["sentiment"])/len(df["sentiment"])

0.6076681407960298

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')
lr.fit(X, target)
#lr.score(X, target)

LogisticRegression(solver='liblinear')

In [12]:
lr.score(X,target)

0.8518600044023773

In [13]:
lr.score(X_dev,target_dev)

0.8410364145658263